In [1]:
import requests
import xmltodict
import time
import pandas as pd
from datetime import datetime

url_base = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson"
url_serviceKey = "S8%2Ftx%2BhEP7bZDZI%2By0P1ZKvPuHpx%2BVUKpt6ay8faxnxR%2FTRO9M5UAy8%2BafhJBNVzQG%2Fgwoym2S4Xbe1dUXivUw%3D%3D"
url_pages = "1000" #페이지당열갯수
url_start_date = "20200303" #시작날짜
url_end_date = datetime.today().strftime("%Y%m%d%H%M%S")[:8] #끝날짜
url = url_base + "?serviceKey=" + url_serviceKey + "&pageNo=1&numOfRows=" + url_pages + "&startCreateDt="+ url_start_date + "&endCreateDt=" + url_end_date

In [2]:
req = requests.get(url).content

In [3]:
xmlObject = xmltodict.parse(req)
dict_data = xmlObject['response']['body']['items']['item']

In [4]:
df = pd.DataFrame(dict_data)

In [5]:
df.drop(['createDt', 'seq', 'stateTime', 'updateDt'], axis=1, inplace=True)

In [6]:
df['newDecideCnt'] = 0

for i in range(len(df)-1):
    df['newDecideCnt'][i] = int(df.iloc[i]['decideCnt']) - int(df.iloc[i+1]['decideCnt'])

<ipython-input-6-65956fc713e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['newDecideCnt'][i] = int(df.iloc[i]['decideCnt']) - int(df.iloc[i+1]['decideCnt'])


- accDefRate : 누적 확진률
- accExamCnt : 누적 검사수
- accExamCompCnt : 누적 검사 완료 수
- careCnt : 치료 중 환자 수
- clearCnt : 격리 해제 수
- deathCnt : 사망자 수
- decideCnt : 확진자 수
- examCnt : 검사 진행 수
- resutlNegCnt : 결과 음성 수
- stateDt : 기준일
- newDecideCnt : 당일 확진자 수

In [7]:
df

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,deathCnt,decideCnt,examCnt,resutlNegCnt,stateDt,newDecideCnt
0,2.5445349277,15905189,14461346,26153,338947,2874,367974,1443843,14093372,20211102,1589
1,2.5398949325,15849303,14425203,26174,337353,2858,366385,1424100,14058818,20211101,1685
2,2.5296344210,15804065,14417103,25303,336548,2849,364700,1386962,14052403,20211031,2061
3,2.5167069578,15775700,14409266,25228,334581,2830,362639,1366434,14046627,20211030,2104
4,2.5085768649,15730785,14372093,24723,332995,2817,360535,1358692,14011558,20211029,2124
...,...,...,...,...,...,...,...,...,...,...,...
604,4.267542836,178189,158569,6608,115,44,6767,19620,151802,20200307,483
605,4.397234584,164740,142908,6136,106,42,6284,21832,136624,20200306,518
606,4.622748154,146541,124731,5645,86,35,5766,21810,118965,20200305,438
607,4.919985595,136707,108293,5256,40,32,5328,28414,102965,20200304,516


In [8]:
df.info

<bound method DataFrame.info of        accDefRate accExamCnt accExamCompCnt careCnt clearCnt deathCnt  \
0    2.5445349277   15905189       14461346   26153   338947     2874   
1    2.5398949325   15849303       14425203   26174   337353     2858   
2    2.5296344210   15804065       14417103   25303   336548     2849   
3    2.5167069578   15775700       14409266   25228   334581     2830   
4    2.5085768649   15730785       14372093   24723   332995     2817   
..            ...        ...            ...     ...      ...      ...   
604   4.267542836     178189         158569    6608      115       44   
605   4.397234584     164740         142908    6136      106       42   
606   4.622748154     146541         124731    5645       86       35   
607   4.919985595     136707         108293    5256       40       32   
608   5.329139718     125851          90296    4750       34       28   

    decideCnt  examCnt resutlNegCnt   stateDt  newDecideCnt  
0      367974  1443843     14

In [27]:
import requests
import xmltodict
import time
import pandas as pd
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


url_decide_base = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson"
url_decide_serviceKey = "S8%2Ftx%2BhEP7bZDZI%2By0P1ZKvPuHpx%2BVUKpt6ay8faxnxR%2FTRO9M5UAy8%2BafhJBNVzQG%2Fgwoym2S4Xbe1dUXivUw%3D%3D"

url_district_base = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson"
url_district_serviceKey = "S8%2Ftx%2BhEP7bZDZI%2By0P1ZKvPuHpx%2BVUKpt6ay8faxnxR%2FTRO9M5UAy8%2BafhJBNVzQG%2Fgwoym2S4Xbe1dUXivUw%3D%3D"

url_seoul_vaccine = 'http://openapi.seoul.go.kr:8088/706e775a4577686136356f44596267/xml/tvCorona19VaccinestatNew/1/999/'

url_vaccine = 'https://api.odcloud.kr/api/15077756/v1/vaccine-stat?page=1&perPage=3000&serviceKey=S8%2Ftx%2BhEP7bZDZI%2By0P1ZKvPuHpx%2BVUKpt6ay8faxnxR%2FTRO9M5UAy8%2BafhJBNVzQG%2Fgwoym2S4Xbe1dUXivUw%3D%3D'


In [25]:
url_pages = "1000" #페이지당열갯수
url_start_date = "20200303" #시작날짜
url_end_date = datetime.today().strftime("%Y%m%d%H%M%S")[:8] #끝날짜
url = url_decide_base + "?serviceKey=" + url_decide_serviceKey + "&pageNo=1&numOfRows=" + url_pages + "&startCreateDt="+ url_start_date + "&endCreateDt=" + url_end_date

req = requests.get(url).content
print(type(req))

xmlObject = xmltodict.parse(req)
print(type(xmlObject))

dict_data = xmlObject['response']['body']['items']['item']
print(type(dict_data))

<class 'bytes'>
<class 'collections.OrderedDict'>
<class 'list'>


In [15]:
url_pages = "1000" #페이지당열갯수
url_start_date = "20200303" #시작날짜
url_end_date = datetime.today().strftime("%Y%m%d%H%M%S")[:8] #끝날짜
url = url_decide_base + "?serviceKey=" + url_decide_serviceKey + "&pageNo=1&numOfRows=" + url_pages + "&startCreateDt="+ url_start_date + "&endCreateDt=" + url_end_date

req = requests.get(url).content

xmlObject = xmltodict.parse(req)
dict_data = xmlObject['response']['body']['items']['item']

dfDecide = pd.DataFrame(dict_data)

dfDecide.drop(['createDt', 'seq', 'stateTime', 'updateDt', 'resutlNegCnt'], axis=1, inplace=True)
dfDecide.sort_values(['stateDt'], ascending=False, inplace=True)
dfDecide['stateDt'] = pd.to_datetime(dfDecide['stateDt']) - pd.DateOffset(days=1)
dfDecide = dfDecide.astype({"accExamCnt":"int64", "accExamCompCnt":"int64", "careCnt":"int64", "clearCnt":"int64", "deathCnt":"int64", "decideCnt":"int64", "examCnt":"int64", "accDefRate":"float64"}).copy()

dfDecide['newDecideCnt'] = 0

for i in range(len(dfDecide)-1):
    dfDecide['newDecideCnt'][i] = int(dfDecide.iloc[i]['decideCnt']) - int(dfDecide.iloc[i+1]['decideCnt'])
    
dfDecide = dfDecide[['stateDt', 'newDecideCnt', 'examCnt', 'decideCnt', 'deathCnt', 'clearCnt', 'careCnt', 'accDefRate', 'accExamCnt', 'accExamCompCnt']].copy()
dfDecide.rename(columns={'stateDt':'날짜', 'newDecideCnt':'당일 확진자 수', 'examCnt':'검사 진행 수', 'decideCnt':'누적 확진자 수', 'deathCnt':'누적 사망자 수', 'clearCnt':'누적 격리 해제 수', 
                         'careCnt':'치료 중 환자 수', 'accDefRate':'누적 확진률', 'accExamCnt':'누적 검사 수', 'accExamCompCnt':'누적 검사 완료 수'}, inplace=True)

dfDecide

,날짜,당일 확진자 수,검사 진행 수,누적 확진자 수,누적 사망자 수,누적 격리 해제 수,치료 중 환자 수,누적 확진률,누적 검사 수,누적 검사 완료 수
0,2021-11-01,1589,1443843,367974,2874,338947,26153,2.544535,15905189,14461346
1,2021-10-31,1685,1424100,366385,2858,337353,26174,2.539895,15849303,14425203
2,2021-10-30,2061,1386962,364700,2849,336548,25303,2.529634,15804065,14417103
3,2021-10-29,2104,1366434,362639,2830,334581,25228,2.516707,15775700,14409266
4,2021-10-28,2124,1358692,360535,2817,332995,24723,2.508577,15730785,14372093
...,...,...,...,...,...,...,...,...,...,...
604,2020-03-06,483,19620,6767,44,115,6608,4.267543,178189,158569
605,2020-03-05,518,21832,6284,42,106,6136,4.397235,164740,142908
606,2020-03-04,438,21810,5766,35,86,5645,4.622748,146541,124731
607,2020-03-03,516,28414,5328,32,40,5256,4.919986,136707,108293


In [16]:
url_district= url_district_base + "?serviceKey=" + url_district_serviceKey + "&pageNo=1&numOfRows=" + url_pages + "&startCreateDt="+ url_start_date + "&endCreateDt=" + url_end_date

req = requests.get(url_district).content

xmlObject = xmltodict.parse(req)
dict_data = xmlObject['response']['body']['items']['item']

dfDistrict = pd.DataFrame(dict_data)
dfDistrict = dfDistrict[(dfDistrict['gubun'] != '합계') & (dfDistrict['gubun'] != '검역')].reset_index(drop=True).copy()

tempDf = pd.DataFrame(columns=['stateDt', 'Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon', 'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do' ,'Jeollabuk-do' ,'Jeollanam-do' ,'Gyeongsangbuk-do' ,'Gyeongsangnam-do' ,'Jeju'])

for i in range(0, len(dfDistrict), 17):
    tempDistrict = dfDistrict.iloc[i:i+17]
    temp = tempDistrict[['incDec', 'stdDay']].transpose()
    yearData = int(temp[i]['stdDay'].split(' ')[0][:-1])
    monthData = int(temp[i]['stdDay'].split(' ')[1][:-1])
    dayData = int(temp[i]['stdDay'].split(' ')[2][:-1])
    
    temp.columns = dfDistrict['gubunEn'][:17]
    temp['stateDt'] = pd.to_datetime(datetime(yearData, monthData, dayData).strftime("%Y-%m-%d")) - pd.DateOffset(days=1)
    temp.head(1)
    temp = temp.head(1)[['stateDt', 'Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon', 'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do' ,'Jeollabuk-do' ,'Jeollanam-do' ,'Gyeongsangbuk-do' ,'Gyeongsangnam-do' ,'Jeju']].copy()
    
    tempDf = tempDf.append(temp)
    
tempDf.reset_index(drop=True, inplace=True)
tempDf = tempDf.astype('int64')
tempDf = tempDf.astype({"stateDt":"datetime64[ns]"})

tempDf.rename(columns={'stateDt':'날짜', 'Seoul':'서울', 'Busan':'부산', 'Daegu':'대구', 'Incheon':'인천', 'Gwangju':'광주', 'Daejeon':'대전', 'Ulsan':'울산', 'Sejong':'세종',
                           'Gyeonggi-do':'경기', 'Gangwon-do':'강원', 'Chungcheongbuk-do':'충북', 'Chungcheongnam-do':'충남', 'Jeollabuk-do':'전북',
                           'Jeollanam-do':'전남', 'Gyeongsangbuk-do':'경북', 'Gyeongsangnam-do':'경남', 'Jeju':'제주'}, inplace=True)

tempDf

,날짜,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주
0,2021-11-01,600,28,42,113,17,28,7,5,501,29,24,69,14,34,21,54,2
1,2021-10-31,646,55,57,127,5,9,12,2,564,13,21,47,26,18,27,50,2
2,2021-10-30,760,58,57,157,23,11,5,4,701,23,27,62,21,30,56,61,1
3,2021-10-29,706,45,90,123,16,7,3,4,719,25,46,56,26,12,40,173,9
4,2021-10-28,770,43,117,132,5,9,4,1,762,21,35,68,56,13,42,34,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,2020-03-06,0,5,65,0,0,2,5,1,10,1,0,0,0,0,390,1,3
605,2020-03-05,0,3,122,0,0,4,3,2,10,0,0,2,0,0,367,3,2
606,2020-03-04,1,9,81,0,0,4,1,2,7,0,0,1,0,1,326,0,5
607,2020-03-03,0,2,95,0,0,1,0,1,9,0,3,1,2,1,398,3,0


In [17]:
dfDistrict.drop(['stdDay', 'updateDt', 'seq', 'qurRate', 'gubunCn'], axis=1, inplace=True)

dfDistrict = dfDistrict.astype({"createDt":"datetime64[ns]"}).copy()

dfDistrict['createDt'] = dfDistrict['createDt'] - pd.DateOffset(days=1)
dfDistrict['stateDt'] = dfDistrict['createDt'].dt.date         # YYYY-MM-DD(문자)
dfDistrict.drop(['createDt'], axis=1, inplace=True)

dfDistrict = dfDistrict.astype({"stateDt":"datetime64[ns]", "defCnt":"int64", "deathCnt":"int64", "incDec":"int64"}).copy()
dfDistrict = dfDistrict[['stateDt', 'gubun', 'gubunEn', 'defCnt', 'deathCnt', 'incDec', 'isolClearCnt', 'isolIngCnt', 'localOccCnt', 'overFlowCnt']].copy()

dfDistrict['isolClearCnt'] = dfDistrict['isolClearCnt'].fillna(0).astype("int64")
dfDistrict['isolIngCnt'] = dfDistrict['isolIngCnt'].fillna(0).astype("int64")
dfDistrict['localOccCnt'] = dfDistrict['localOccCnt'].fillna(0).astype("int64")
dfDistrict['overFlowCnt'] = dfDistrict['overFlowCnt'].fillna(0).astype("int64")

dfDistrict = dfDistrict.astype({"isolClearCnt":"int64", "isolIngCnt":"int64", "localOccCnt":"int64", "overFlowCnt":"int64"}).copy()

dfDistrict.rename(columns={'stateDt':'날짜', 'gubun':'지역명', 'gubunEn':'지역명(En)', 'defCnt':'확진자 수', 'deathCnt':'사망자 수', 'incDec':'전일 대비 증감 수',
                           'isolClearCnt':'격리 해제 수', 'isolIngCnt':'격리 중 환자 수', 'localOccCnt':'지역 발생 수', 'overFlowCnt':'해외 유입 수'}, inplace=True)

dfDistrict

,날짜,지역명,지역명(En),확진자 수,사망자 수,전일 대비 증감 수,격리 해제 수,격리 중 환자 수,지역 발생 수,해외 유입 수
0,2021-11-01,제주,Jeju,3104,2,2,3047,55,2,0
1,2021-11-01,경남,Gyeongsangnam-do,13112,41,54,12508,563,54,0
2,2021-11-01,경북,Gyeongsangbuk-do,9757,95,21,9290,372,21,0
3,2021-11-01,전남,Jeollanam-do,3595,22,34,3403,170,34,0
4,2021-11-01,전북,Jeollabuk-do,5226,66,14,4864,296,12,2
...,...,...,...,...,...,...,...,...,...,...
10348,2020-03-02,전북,Jeollabuk-do,7,0,1,1,6,1,0
10349,2020-03-02,전남,Jeollanam-do,4,0,0,1,3,0,0
10350,2020-03-02,경북,Gyeongsangbuk-do,687,7,59,2,678,59,0
10351,2020-03-02,경남,Gyeongsangnam-do,63,0,1,0,63,1,0


In [18]:
req = requests.get(url_seoul_vaccine).content

xmlObject = xmltodict.parse(req)
dict_data = xmlObject['tvCorona19VaccinestatNew']['row']

dfSeoulVac = pd.DataFrame(dict_data)

dfSeoulVac = dfSeoulVac.astype({"S_VC_DT":"datetime64[ns]"}).copy()

dfSeoulVac = dfSeoulVac.astype({"FIR_SUB":"int64", "FIR_INC":"int64", "FIR_INC_RATE":"float64", "FIR_INC1":"int64", "SCD_INC":"int64", "SCD_INC_RATE":"float64", "SCD_INC1":"int64"}).copy()
dfSeoulVac.rename(columns={'S_VC_DT':'날짜', "FIR_SUB":"접종대상자", "FIR_INC":"누적 1차 접종자 수", "FIR_INC_RATE":"1차 접종률", 
                           "FIR_INC1":"당일 1차 접종자 수", "SCD_INC":"누적 2차 접종자 수", "SCD_INC_RATE":"2차 접종률", "SCD_INC1":"당일 2차 접종자 수"}, inplace=True)
                           
dfSeoulVac

,날짜,접종대상자,누적 1차 접종자 수,1차 접종률,당일 1차 접종자 수,누적 2차 접종자 수,2차 접종률,당일 2차 접종자 수
0,2021-11-01,9505868,7706849,81.1,14487,7288535,76.7,22876
1,2021-10-31,9505868,7691201,80.9,590,7264156,76.4,3541
2,2021-10-30,9505868,7690845,80.9,6786,7260745,76.4,62186
3,2021-10-29,9505868,7684177,80.8,10066,7197005,75.7,119127
4,2021-10-28,9505868,7673374,80.7,9889,7075798,74.4,118168
...,...,...,...,...,...,...,...,...
190,2021-04-25,1023346,350631,34.2,23323,18513,1.8,3411
191,2021-04-24,1023346,350427,34.2,200,18513,1.8,0
192,2021-04-23,1027740,345764,33.6,10195,18376,1.8,119
193,2021-04-22,1023538,317386,31.0,27074,15901,1.6,2479


In [29]:
req = requests.get(url_vaccine).content

df_vaccine = pd.read_json(req)

df_full_vaccine = pd.DataFrame(columns=['accumulatedFirstCnt', 'accumulatedSecondCnt', 'baseDate', 'firstCnt', 'secondCnt', 'sido', 'totalFirstCnt', 'totalSecondCnt'])

for i in range(len(df_vaccine)):
    temp = pd.Series(df_vaccine['data'][i])
    if temp['sido'] == '전국':
        continue
    df_full_vaccine = df_full_vaccine.append(temp, ignore_index=True)

df_full_vaccine.loc[df_full_vaccine.sido == '전라남도', 'sido'] = '전남'
df_full_vaccine.loc[df_full_vaccine.sido == '전라북도', 'sido'] = '전북'
df_full_vaccine.loc[df_full_vaccine.sido == '경상남도', 'sido'] = '경남'
df_full_vaccine.loc[df_full_vaccine.sido == '경상북도', 'sido'] = '경북'
df_full_vaccine.loc[df_full_vaccine.sido == '충청남도', 'sido'] = '충남'
df_full_vaccine.loc[df_full_vaccine.sido == '충청북도', 'sido'] = '충북'

df_full_vaccine.drop(['accumulatedFirstCnt', 'accumulatedSecondCnt'], axis=1, inplace=True)
df_full_vaccine['sido'] = df_full_vaccine['sido'].str[:2]
df_full_vaccine.sort_values('baseDate', ascending=False, inplace=True)
df_full_vaccine.reset_index(drop=True, inplace=True)

df_full_vaccine = df_full_vaccine.astype({"baseDate":"datetime64[ns]", "firstCnt":"int64", "secondCnt":"int64", "totalFirstCnt":"int64", "totalSecondCnt":"int64"}).copy()
df_full_vaccine['baseDate'] = df_full_vaccine['baseDate'] - pd.DateOffset(days=1)
df_full_vaccine = df_full_vaccine[['baseDate', 'sido', 'firstCnt', 'secondCnt', 'totalFirstCnt', 'totalSecondCnt']].copy()
df_full_vaccine.rename(columns={'baseDate':'날짜', 'firstCnt':'1차 접종 수', 'secondCnt':'2차 접종 수', 'sido':'지역명', 'totalFirstCnt':'누적 1차 접종 수', 'totalSecondCnt':'누적 2차 접종 수'}, inplace=True)

df_full_vaccine

,날짜,지역명,1차 접종 수,2차 접종 수,누적 1차 접종 수,누적 2차 접종 수
0,2021-08-23,광주,10368,20631,733429,349501
1,2021-08-23,인천,21150,38955,1423384,625045
2,2021-08-23,대구,19776,29613,1193368,515271
3,2021-08-23,부산,26130,53194,1797360,837341
4,2021-08-23,서울,74785,130725,4856923,2342494
...,...,...,...,...,...,...
2828,2021-03-10,전남,2718,0,25748,0
2829,2021-03-10,경북,2365,0,28435,0
2830,2021-03-10,경남,2036,0,43848,0
2831,2021-03-10,제주,500,0,4830,0
